<a href="https://colab.research.google.com/github/tearicee/Colab/blob/main/00900_3th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import datetime as dt
import os
import chardet
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/指數調整/2 指數調整 00900 3rd_test.csv', encoding='cp950', skiprows=5, header = None)
df.columns = ['market_value_rank', 'stock_id', 'stock_name', 'close', 'return(%)', 'volume', 'past_close', 'total_return(%)', 'market_value', 'trading_money', 'income', 'dividend_yield', 'cash_dividend']
df['market_value_rank'] = df['market_value'].rank(ascending=False).astype(int)
df['stock_id'] = df['stock_id'].str.replace('.TW', '')
df

<ipython-input-17-02367eceeb2b>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['stock_id'] = df['stock_id'].str.replace('.TW', '')


,market_value_rank,stock_id,stock_name,close,return(%),volume,past_close,total_return(%),market_value,trading_money,income,dividend_yield,cash_dividend
0,34,1101,台泥,34.20,1.79,33522,33.90,2.27,2283.57,9.653590e+08,3506.55,2.11,1.00
1,62,1102,亞泥,41.70,0.85,7012,41.30,6.43,1464.47,1.915523e+08,12650.00,7.57,3.40
2,362,1103,嘉泥,19.10,0.79,290,17.75,11.31,137.52,3.692677e+06,-410.42,-0.74,0.65
3,350,1104,環泥,28.15,0.90,803,21.90,39.08,143.14,6.981963e+06,805.84,7.34,1.00
4,794,1108,幸福,15.10,1.00,838,10.90,45.61,44.12,2.314381e+06,506.26,9.11,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652,772,9951,皇田,77.20,-0.64,101,61.40,31.27,45.99,5.230823e+06,334.44,4.71,3.30
1653,1207,9955,佳龍,24.10,1.05,483,19.00,26.84,19.61,6.570273e+06,-109.58,-0.00,0.00
1654,267,9958,世紀鋼,163.50,-1.21,2611,85.00,92.97,197.54,3.000689e+08,617.97,1.95,2.99
1655,1523,9960,邁達康,27.10,-0.37,28,25.75,13.44,8.65,1.920059e+06,141.04,10.36,1.70


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/指數調整/2 指數調整 00900 Q3.csv', encoding='cp950', skiprows=5, header = None)
df.columns = ['market_value_rank', 'stock_id', 'stock_name', 'close', 'return(%)', 'volume', 'market_value', 'trading_money', 'income', 'dividend_yield']
df['stock_id'] = df['stock_id'].str.replace('.TW', '')
df

<ipython-input-127-b3cb34069606>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['stock_id'] = df['stock_id'].str.replace('.TW', '')


,market_value_rank,stock_id,stock_name,close,return(%),volume,market_value,trading_money,income,dividend_yield
0,1,2330,台積電,577.00,-0.52,26606,149628.05,1.497283e+10,986301.87,2.51
1,2,2454,聯發科,912.00,0.88,7093,14588.03,6.826696e+09,65150.04,5.59
2,3,2317,鴻海,101.00,-1.46,29960,14001.62,4.130684e+09,161830.89,4.85
3,4,2412,中華電,118.00,0.85,7939,9153.79,8.710722e+08,46856.01,4.04
4,5,2382,廣達,212.00,1.68,27080,8188.77,1.066537e+10,41683.18,3.67
...,...,...,...,...,...,...,...,...,...,...
1672,1673,2321,東訊,16.70,3.73,5,2.38,1.446319e+05,-10.16,-0.00
1673,1674,4804,大略-KY,4.85,6.36,54,2.20,4.382456e+05,-143.26,-0.00
1674,1675,4712,南璋,2.74,-5.52,159,2.02,2.538181e+05,-9.63,-0.00
1675,1676,6404,通訊-KY,5.15,-9.97,78,1.25,6.235120e+05,-17.36,-0.00


In [3]:
# @title 流動性篩選
df_market_value = df.nlargest(200, 'market_value')

In [4]:
# @title 成交值篩選
df_trading_money = df_market_value.nlargest(180, 'trading_money')
df_trading_money

,market_value_rank,stock_id,stock_name,close,return(%),volume,past_close,total_return(%),market_value,trading_money,income,dividend_yield,cash_dividend
304,305,2330,台積電,585.00,1.39,39881,490.00,21.97,127058.86,2.136941e+10,979010.04,3.94,11.0
580,581,3037,欣興,168.50,5.31,36286,155.50,13.67,2294.03,6.150085e+09,34290.96,4.91,3.4
811,812,3661,世芯-KY,3135.00,-1.57,1811,927.00,239.97,664.50,5.898554e+09,2097.70,1.28,11.8
1480,1481,8069,元太,176.00,-0.85,5145,181.00,-0.68,2064.13,5.853592e+09,7830.64,3.49,3.2
725,726,3443,創意,1740.00,1.46,3914,731.00,140.22,979.63,5.764992e+09,3211.12,2.44,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1415,6592,和潤企業,130.50,1.16,435,102.00,43.96,525.30,4.250052e+07,4836.73,4.39,4.2
9,10,1210,大成,54.50,1.68,2388,44.55,25.92,398.62,4.200443e+07,2854.42,3.77,1.5
19,20,1229,聯華,67.10,2.44,2752,51.10,43.19,756.35,4.040547e+07,1661.77,3.35,1.8
176,177,1717,長興,29.70,-0.34,2379,33.25,-7.25,412.39,4.028185e+07,3430.27,4.70,1.5


In [5]:
# @title 營業利益篩選
df_income = df_trading_money[df_trading_money['income'] > 0]

In [6]:
# @title 股利率排序
df_yield = df_income.nlargest(30, 'dividend_yield')

In [7]:
# @title 權重計算1
df_yield['weighted_dividend_yield'] = df_yield['dividend_yield'] * df_yield['market_value']
# 調整權重
df_yield['adjusted_weight'] = df_yield['weighted_dividend_yield'] / df_yield['weighted_dividend_yield'].sum()
df_yield['final_weight'] = df_yield['adjusted_weight'].apply(lambda x: min(max(x, 0.01), 0.05)) * 100

In [8]:
# @title 權重計算2
df_index = df_yield
# 計算每個成分股的新權重
df_index['new_weight'] = df_index['final_weight'].apply(lambda x: min(max(x, 1), 5))

# 計算剩餘的權重
remaining_weight = 100 - df_index['new_weight'].sum()

# 計算需要分配給原始權重不足1%的股票的平均權重
average_weight = remaining_weight / (df_index['final_weight'] < 5).sum()

# 將剩餘的權重平均分配給原始權重不足1%的股票
df_index.loc[df_index['final_weight'] < 5, 'new_weight'] += average_weight

# 顯示結果
print(df_index[['stock_id', 'stock_name', 'close', 'new_weight']])
print("總持有部位：", df_index['new_weight'].sum())

     stock_id stock_name   close  new_weight
473      2637      慧洋-KY   44.45    5.000000
225      2002         中鋼   25.70    5.000000
465      2615         萬海   46.55    5.000000
454      2603         長榮  113.50    5.000000
577      3034         聯詠  492.50    4.898783
459      2609         陽明   41.90    5.203198
456      2606         裕民   45.00    2.381487
679      3293         鈊象  638.00    2.381487
390      2454        聯發科  926.00    5.000000
322      2357         華碩  389.00    4.298522
1637     9933         中鼎   42.90    2.381487
659      3231         緯創  102.50    2.547442
290      2303         聯電   50.30    5.000000
386      2449       京元電子   82.70    2.381487
1215     6147         頎邦   69.80    2.381487
313      2347         聯強   68.80    2.606335
841      3711      日月光投控  127.00    5.000000
1086     5347         世界   78.70    3.056868
354      2404         漢唐  245.50    2.381487
1440     6670       復盛應用  204.00    2.381487
339      2379         瑞昱  430.50    3.298910
836      3

In [9]:
# @title 買超張數 & 量能占比
net_value = 36901250921
df_index['holding_market_value'] = net_value * df_index['new_weight']
df_index['buy_unit'] = np.round(df_index['holding_market_value'] / df_index['close'] / 1000 / 100, 0)
df_index['unit_volume_ratio'] = df_index['buy_unit'] / df_index['volume']
df_index = df_index.sort_values(by = ['unit_volume_ratio'], ascending = False)
df_index[['stock_id', 'stock_name', 'new_weight', 'buy_unit', 'unit_volume_ratio' ]]

,stock_id,stock_name,new_weight,buy_unit,unit_volume_ratio
1440,6670,復盛應用,2.381487,4308.0,45.829787
425,2504,國產,2.381487,33162.0,20.713304
473,2637,慧洋-KY,5.000000,41509.0,19.803912
1637,9933,中鼎,2.381487,20485.0,9.626410
465,2615,萬海,5.000000,39636.0,7.363180
1215,6147,頎邦,2.381487,12590.0,6.790723
307,2337,旺宏,2.381487,30674.0,5.563940
456,2606,裕民,2.381487,19529.0,5.328513
459,2609,陽明,5.203198,45824.0,4.913048
1295,6239,力成,2.381487,8213.0,4.415591


In [10]:
import requests
from bs4 import BeautifulSoup

url = 'https://websys.fsit.com.tw/FubonETF/Fund/Assets.aspx?stkId=00900'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 下載網頁內容
response = requests.get(url, headers = headers)
html = response.text

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html, 'html.parser')

# 找到所有class為 "mb60 current content1_1_1" 的元素
elements = soup.find_all('div', class_='mb60 current content1_1_1')

# 確保有兩個元素
if len(elements) >= 2:
    # 取得第二個元素
    second_element = elements[1]

    # 找到表格元素
    table = second_element.find('table')

    # 印出表格HTML
    #print(table.prettify())
else:
    print("找不到足夠的元素")

data = []
for row in table.find_all('tr')[1:]:
    row_data = [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

# 創建 DataFrame
df_900 = pd.DataFrame(data, columns = ['stock_id', 'stock_name', 'holdings', 'market_value', 'ratio(%)'])[:-1]
df_900['holdings'] = df_900['holdings'].str.replace(',',"")
df_900['holdings'] = df_900['holdings'].astype(int) / 1000
df_900

,stock_id,stock_name,holdings,market_value,ratio(%)
0,3702,大聯大,29147.000,"2,244,319,000",6.1251
1,2454,聯發科,2394.000,"2,224,026,000",6.0697
2,2382,廣達,10835.000,"2,112,825,000",5.7662
3,2379,瑞昱,4250.000,"1,899,750,000",5.1847
4,2891,中信金,66271.000,"1,772,749,250",4.8381
5,6121,新普,4185.000,"1,757,700,000",4.797
6,2474,可成,8601.000,"1,664,293,500",4.5421
7,2886,兆豐金,41762.088,"1,626,633,328",4.4393
8,1216,統一,21584.000,"1,597,216,000",4.359
9,3008,大立光,658.000,"1,566,040,000",4.274


##納入00900名單

In [20]:
df_in = df_index[ ~(df_index['stock_id'].isin(df_900['stock_id'])) ][['stock_id', 'stock_name', 'new_weight', 'buy_unit', 'unit_volume_ratio']]
df_in.reset_index(drop = True)

,stock_id,stock_name,new_weight,buy_unit,unit_volume_ratio
0,6670,復盛應用,2.381487,4308.0,45.829787
1,2504,國產,2.381487,33162.0,20.713304
2,2637,慧洋-KY,5.000000,41509.0,19.803912
3,9933,中鼎,2.381487,20485.0,9.626410
4,2615,萬海,5.000000,39636.0,7.363180
5,6147,頎邦,2.381487,12590.0,6.790723
6,2337,旺宏,2.381487,30674.0,5.563940
7,2606,裕民,2.381487,19529.0,5.328513
8,2609,陽明,5.203198,45824.0,4.913048
9,6176,瑞儀,2.381487,6866.0,3.751913


##維持名單

In [19]:
df_hold = df_index[ (df_index['stock_id'].isin(df_900['stock_id'])) ][['stock_id', 'stock_name', 'new_weight', 'buy_unit', 'unit_volume_ratio']]
df_hold.reset_index(drop = True)

,stock_id,stock_name,new_weight,buy_unit,unit_volume_ratio
0,6239,力成,2.381487,8213.0,4.415591
1,2915,潤泰全,2.381487,14266.0,4.101783
2,3293,鈊象,2.381487,1377.0,3.209790
3,6121,新普,2.381487,2418.0,2.427711
4,2379,瑞昱,3.298910,2828.0,1.421820
5,2002,中鋼,5.000000,71792.0,1.296915
6,2454,聯發科,5.000000,1993.0,0.199420


##剔除00900名單

In [18]:
df_out = df_900[(df_900['stock_id'].isin(df_hold['stock_id']))]
df_out = df_out.merge( df, left_on = ['stock_id', 'stock_name'], right_on =['stock_id', 'stock_name'] )[['stock_id', 'stock_name', 'market_value_rank', 'volume', 'holdings']]
df_out['sell_volume_ratio'] = df_out['holdings'] / df_out['volume']
df_out.sort_values(by = ['sell_volume_ratio'], ascending = False)

,stock_id,stock_name,market_value_rank,volume,holdings,sell_volume_ratio
4,6239,力成,109,1860,10757.0,5.783333
2,6121,新普,119,996,4185.0,4.201807
5,3293,鈊象,125,429,1324.0,3.086247
1,2379,瑞昱,54,1989,4250.0,2.136752
6,2915,潤泰全,133,3478,5471.0,1.573030
3,2002,中鋼,13,55356,48812.0,0.881783
0,2454,聯發科,3,9994,2394.0,0.239544
